In [2]:
import sys

In [511]:
# author: Dan Dean, 966066212
import sys
LETTERS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
DIGITS = "0123456789"
METACHARS = "</bodyuliEOI" #we don't want the > so that we dont accidently take 
TAGS = ["<b>","<i>","<body>","<li>","<ul>"]
ENDTAGS = ["</b>","</i>","</body>","</li>","</ul>","<EOI>"]
TAGSMATCH = {
    "<b>": "</b>"
    ,"<i>" :"</i>"
    ,"<body>":"</body>"
    ,"<li>":"</li>"
    ,"<ul>" : "</ul>"
}


In [344]:
class Parser:
    def __init__(self, s):
        self.lexer = Lexer(s+"$")
        self.token = self.lexer.nextToken()
        self.level = 0
    
    def toList(self):
        words = []
        alexis = Lexer(s)
        while True:
            tok = alexis.nextToken()
            words.append(tok)
            if tok == "<EOI>":
                break
        return words
        
    def run(self):
        self.statement()
        
    def statement(self):
        print("<Statement>")
        self.assignmentStmt()
        while self.token.getTokenType() == SEMICOLON:
            print("\t<Semicolon>;</Semicolon>")
            self.token = self.lexer.nextToken()
            self.assignmentStmt()
        self.match(EOI)
        print("</Statement>")
        
    def assignmentStmt(self):
        print("\t<Assignment>")
        val = self.match(ID)
        print("\t\t<Identifier>" + val + "</Identifier>")
        self.match(ASSIGNMENTOP)
        print("\t\t<AssignmentOp>:=</AssignmentOp>")
        self.expression()
        print("\t</Assignment>")
        
    def expression(self):
        if self.token.getTokenType() == ID:
            print("\t\t<Identifier>" + self.token.getTokenValue() \
                   + "</Identifier>")
        elif self.token.getTokenType() == INT:
            print("\t\t<Int>" + self.token.getTokenValue() + "</Int>")
        elif self.token.getTokenType() == FLOAT:
            print("\t\t<Float>" + self.token.getTokenValue() + "</Float>")
        else:
            print("Syntax error: expecting an ID, an int, or a float" \
                  + "; saw:" \
                  + typeToString(self.token.getTokenType()))
            sys.exit(1)
        self.token = self.lexer.nextToken()
        
    def match(self, tp):
        val = self.token.getTokenValue()
        if (self.token.getTokenType() == tp):
            self.token = self.lexer.nextToken()
        else: self.error(tp)
        return val
    
    def error(self, tp):
        print ("Syntax error: expecting: " + typeToString(tp) \
               + "; saw: " + typeToString(self.token.getTokenType()))
        sys.exit(1)
        

In [345]:
def typeToString (tp):
    if (tp == INT): return "Int"
    elif (tp == FLOAT): return "Float"
    elif (tp == ID): return "ID"
    elif (tp == SEMICOLON): return "Semicolon"
    elif (tp == ASSIGNMENTOP): return "AssignmentOp"
    elif (tp == EOI): return "EOI"
    return "Invalid"


In [437]:
class Lexer:
    # stmt is the current statement to perform the lexing;
    # index is the index of the next char in the statement
    def __init__ (self, s):
        self.stmt = s + "<EOI>"
        self.index = 0
        self.nextChar()
        
    def nextToken(self):
        while 1:
            if self.checkChar("<"):
                return (self.consumeChars(METACHARS))
            elif self.ch in LETTERS+DIGITS:
                return (self.consumeChars(LETTERS+DIGITS))
            else:
                self.nextChar()
            
    def nextChar(self):
        self.ch = self.stmt[self.index] 
        self.index = self.index + 1
        if self.index >= len(self.stmt):
            self.index = 0
            
    def consumeChars (self, charSet):
        r = self.ch
        self.nextChar()
        while (self.ch in charSet):
            r = r + self.ch
            self.nextChar()
        if charSet == METACHARS:
            r += ">"
            if self.index < len(self.stmt):
                self.nextChar()
        return r
    
    def checkChar(self, c):
        if (self.ch==c):
            return True
        return False

In [468]:
class Token:
    "A class for representing Tokens"
    def __init__ (self, s):
        self.stmt = s
        self.type = "INVALID"
        self.type = self.typer()
        self.level = 0
        
    def getToken(self):
        return self.stmt
    
    def getType(self):
        return self.type
    
    def typer(self):
        if self.isTag():
            if self.stmt in ENDTAGS:
                if self.stmt == "<EOI>":
                    return "EOI"
                return  "ENDTAG"
            else:
                return  "TAG"
        else:
            for i in self.stmt:
                if i not in LETTERS+DIGITS:
                    return "INVALID"
            return "TEXT"
    
    def isTag(self):
        return self.stmt in TAGS + ENDTAGS
    
    def isType(self, check):
        if self.type == check: return True
        return False
    
    def __repr__(self):
        return self.type + " token: " + self.stmt 
            

In [516]:
class ParserTwo:
    """| WEBPAGE -> <body> { TEXT } </body>
       | TEXT -> STRING | <b> TEXT </b> | <i> TEXT </i> | <ul> { LISTITEM } </ul>
       | LISTITEM -> <li> TEXT </li>
       | STRING -> LETTER | DIGIT    """
    def __init__(self, s):
        self.lexer = Lexer(s)
        self.tokenList = self.toList()
        self.token = self.tokenList[0]
        self.cur_level = 0
        
    def toList(self):
        words = []
        while True:
            tok = self.lexer.nextToken()
            words.append(Token(tok))
            if tok == "<EOI>":
                break
        return words
        
    def parse(self):
        last_tag = self.tokeList[0]
        for tok in self.tokenList:
            if tok.getType() == "TAG":
                print("   "*self.cur_level + tok.getToken())
                self.cur_level +=1
                last_tag = tok
            elif tok.getType() == "ENDTAG":
                
                self.cur_level -= 1
                print("   "*self.cur_level + tok.getToken())
            elif tok.getType() == "EOI":
                print("done")
            else:
                print("   "*self.cur_level + tok.getToken())
                
    def run(self):
        self.parse()
        

In [496]:
PT = ParserTwo(("<body> google <b><i><ul> yahoo</ul></i></b></body>"))

In [497]:
PT.run()

<body>
   google
   <b>
      <i>
         <ul>
            yahoo
         </ul>
      </i>
   </b>
</body>
done


In [498]:
PT.tokenList

[TAG token: <body>,
 TEXT token: google,
 TAG token: <b>,
 TAG token: <i>,
 TAG token: <ul>,
 TEXT token: yahoo,
 ENDTAG token: </ul>,
 ENDTAG token: </i>,
 ENDTAG token: </b>,
 ENDTAG token: </body>,
 EOI token: <EOI>]

In [510]:
stack = []
for tok in PT.tokenList: 
    if tok.isType("TAG") :
        stack.append(tok)
    if tok.isType("ENDTAG"):
        compIndex = ENDTAGS.index(tok.getToken())
        print(tok,"->",TAGS[compIndex])

ENDTAG token: </ul> -> <ul>
ENDTAG token: </i> -> <i>
ENDTAG token: </b> -> <b>
ENDTAG token: </body> -> <body>


In [506]:
stack = [1,2,3]

In [486]:
stack.pop()

3